In [ ]:
from DistMLIP.implementations.mace import MACECalculator_Dist
from mace.calculators import mace_mp

import matgl
from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor

In [ ]:
# Insert your atoms
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(3.5), ["Li", "Mn"], [[0, 0, 0], [0.5, 0.5, 0.5]])
struct.perturb(0.5)
struct.make_supercell((30, 30, 30))
print(f"There are {len(struct)} atoms.")
atoms = AseAtomsAdaptor().get_atoms(struct)

In [ ]:
# Load/create a previously trained MACECalculator on cpu
from mace.calculators import mace_mp

calc = mace_mp(model="small", device="cpu", enable_cueq=True)


# Make a distributed version of the calculator 
dist_calc = MACECalculator_Dist.from_existing(calc)
dist_calc.models[0].time_dict = {}
# Enable distributed mode and specify the GPUs to use
dist_calc.enable_distributed_mode([0, 1])


In [ ]:
# You can use MACECalculator_Dist the same way as MACECalculator. An example using DistMLIP's MolecularDynamics class:
from DistMLIP.implementations.matgl import MolecularDynamics

driver = MolecularDynamics(
    atoms,
    potential=dist_calc,
    timestep=0.5,
    temperature=300,
    loginterval=200
)

driver.run(20)